# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
# Example of One-Shot Prompting. 
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/tencent/Hunyuan-MT-7B',
 '/tencent/HunyuanWorld-Voyager',
 '/microsoft/VibeVoice-1.5B',
 '/google/embeddinggemma-300m',
 '/moonshotai/Kimi-K2-Instruct-0905',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/wornoutLizard7/Find-Dream-Match',
 '/spaces/Arial311/DeepSite-Gallery',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/apple/fastvlm-webgpu',
 '/spaces',
 '/datasets/HuggingFaceM4/FineVision',
 '/datasets/HuggingFaceFW/finepdfs',
 '/datasets/data-agents/jupyter-agent-dataset',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/syncora/developer-productivity-simulated-behavioral-data',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
tencent/Hunyuan-MT-7B
Updated
about 2 hours ago
•
4.18k
•
543
tencent/HunyuanWorld-Voyager
Updated
4 days ago
•
3.55k
•
480
microsoft/VibeVoice-1.5B
Updated
7 days ago
•
231k
•
1.55k
google/embeddinggemma-300m
Updated
4 days ago
•
3

In [26]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [28]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\ntencent/Hunyuan-MT-7B\nUpdated\nabout 2 hours ago\n•\n4.18k\n•\n544\ntencent/HunyuanWorld-Voyager\nUpdated\n4 days ago\n•\n3.55k\n•\n480\nmicrosoft/VibeVoice-1.5B\nUpdated\n7 days ago\n•\n231k\n•\n1.55k\ngoogle/embeddinggemma-300m\nUpdated\n4 days ago\n•\n35.5k\n•\n399\nmoonshotai/Kimi-K2-Instruct-0905\nUpdated\n3 days ago\n•\n3.84k\n•\n284\nBrowse 1M+ models\nSpaces\nRunning\n12.9k\n12.9k

In [29]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [30]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face: Embrace the Future of AI (and your next hug)!

## What's in the Brochure?
- **A Warm Welcome!**
- **AI Models for Every Mood**
- **Spaces: Where Ideas Sparkle**
- **Join the Hugging Family!**
  
---
  
### A Warm Welcome! 🌍
Welcome to Hugging Face – where we believe that AI should be as cuddly and heartwarming as its name suggests! Our mission? To build the future together, one hug at a time (or maybe a billion lines of code)! The AI community isn't just a place for algorithms; it's where **human connection** meets **machine learning**.

### AI Models for Every Mood 🤖
With over **1 million models** to choose from, you can find the perfect AI companion for every occasion! Whether you want to generate a touching love poem or draft the next great American novel, we’ve got you covered! 

Some trending models this week:
- **tencent/Hunyuan-MT-7B:** Need a linguist? This model speaks a million languages (almost).
- **microsoft/VibeVoice-1.5B:** Want your voice to vibrate with excitement? This model can help you find your pitch!
- **google/embeddinggemma-300m:** Because who doesn’t need an AI named Gemma backing them up?

### Spaces: Where Ideas Sparkle 🌟
In our **Spaces**, you can let your creativity run wild! Imagine trying to create your soulmate with our "Find Dream Match" app, or maybe generating a video *from an image with a text prompt* (because why not turn your sister's graduation photo into an epic action film?). With **12.9k unique applications** running, you're bound to find something that tickles your fancy!

### Join the Hugging Family! 🎉
Want to become part of our world? We’re not just looking for tech maestros; we want passionate individuals from all walks of life. We're like a family, just without the awkward holiday dinners! From engineers to dreamers, if you're looking to jump onto the **AI spaceship**, we have roles with room for your brilliance:

- **Just Got Hired?** Blink twice if you’re ready to bring your creativity and enthusiasm to our open-source community.
- **Tired of Traditional Workplaces?** So are we! Our culture embraces fun and innovation, with the perfect blend of seriousness and silliness (because who can resist a good pun?). 

### The Closing Hug! 🤗
Ready to get your AI hugs? Dive into the ***huggingface.co*** portal today, and let’s build the future together. Embrace the possibilities, and who knows, you might even find your next AI buddy (or a new best friend).

### Connect With Us!
- **[GitHub](https://github.com/huggingface)**
- **[Twitter](https://twitter.com/huggingface)**
- **[LinkedIn](https://linkedin.com/company/huggingface)**
- **[Discord](https://discord.com/invite/huggingface)**

**Join the Hugging Face family today; you won’t regret it! And hey, if you need a hug, we’re just a click away!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [31]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [32]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# 🤗 Welcome to Hugging Face! 🤗

**The AI Community Building the Future (One Hug at a Time!)**  

---

## Who Are We?

At Hugging Face, we’re not just any AI community—we’re the *Hugging* Face community! We’re about making machine learning (ML) as friendly and approachable as your favorite sweater. Whether you are a developer, a researcher, or just an overly enthusiastic cat meme creator, there’s a space for you here! 

We’ve curated over **1 million models** and **250,000 datasets**. Yup, that’s right! We have more models than a fashion week runway, and we’re dressed to impress!

---

## Come Hug Us! Community Vibes & Collaborations

We're all about collaboration—minus the awkward handshakes! Our platform is your playground where enthusiasts can create, discover, and collaborate on public models, datasets, and applications. Imagine a cozy online café where the topic of conversation is AI, and the only thing on the menu is winning!

### Our Fans
- **AI at Meta**: Followed by over 7.63k eager minds!
- **Microsoft**: Sporting a whopping 425 models. 
- And let’s not forget **Amazon**: Only 20 models, but hey, they still deliver in 2 days!

---

## Careers at Hugging Face: The Only Place Where You Get Paid for Embraces!

Thinking of joining us? We’re not just looking for any run-of-the-mill AI enthusiasts; we want the dreamers, the doers, and the ones who provide the best hugs! Here are some perks of joining our team:

- **Innovative Culture**: No boring meetings! Our idea of a brainstorming session involves a mini-robot stimulating thoughts while you sip on your green tea.
- **Flexible Work Hours**: Work when you’re most productive! (Which is usually when you’re in your pajamas, right?)
- **Epic Growth**: With unlimited resources at your clawing fingers, watch your skills skyrocket faster than an AI model after a good dataset!

### Current Openings:
- Hugging Wizard (aka Data Scientist)
- Model Maestro (Data Engineer)
- Hug Specialist (Community Manager)

---

## Why Choose Hugging Face?

- **Discover AI Like Never Before**: With applications for everything from generating dream dates to real-time video captioning,pushing boundaries has never felt so comfy!
- **Open Source Freaks Unite**: We’ve got state-of-the-art tools like Transformers and Diffusers. Join the rebellion against closed-source drudgery.
- **Affordable Pricing**: Start deploying your dreams with Compute and Enterprise solutions starting at just **$0.60/hour for GPUs**!

---

## Join the Hugging Face Family!

Whether you're looking for the next big breakthrough, trying to find your long-lost twin in pixels, or just want to be part of an online family that's all about sharing hugs (virtually, of course!), Hugging Face is the place for you!

Sign up today, and let's shape the future of AI together—one hug at a time!  

For more information, visit us at **[Hugging Face](https://huggingface.co)**.

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}]}



# Hugging Face Brochure

## About Us
**Hugging Face** is at the forefront of the AI community, dedicated to building the future through collaboration on machine learning models, datasets, and applications. Our mission is to democratize machine learning technology and enable users to create, discover, and cooperate effectively on our platform.

## Our Products
- **Models:** Access to over 1 million models, ranging from state-of-the-art AI solutions for various tasks to community-contributed options.
- **Datasets:** Browse through 250k+ datasets tailored for diverse ML tasks, aiding research and development.
- **Spaces:** A collaborative environment with over 400k applications enabling users to run and create applications effortlessly.

## Community
Hugging Face is more than just a platform; it is a thriving community of over 50,000 organizations, including giants like Google, Microsoft, and Amazon. We encourage users to share their work, get feedback, and build their portfolios while advancing the entire machine learning landscape together.

## Company Culture
Our culture promotes openness, collaboration, and innovation. We believe in the power of open-source and encourage contributions from our community to ensure tools are accessible for everyone. Our users range from independent researchers to large enterprises, all passionate about advancing AI technology.

## Careers
Join the Hugging Face team and become part of a mission-driven organization that is catalyzing advances in AI and ML. We are always on the lookout for talented individuals who align with our values of collaboration and openness. Explore current job openings on our [Careers Page](https://huggingface.co/jobs) and find out how you can contribute to our shared mission.

## Work with Us
Our enterprise solutions support teams with advanced platforms, enterprise-grade security, and dedicated support, ensuring successful implementation of AI solutions in various organizational setups.

---

## Connect with Us
- **Website:** [HuggingFace.co](https://huggingface.co)
- **Community Forums:** Join discussions and connect with fellow ML enthusiasts.
- **Social Media:** Follow us on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), and [GitHub](https://github.com/huggingface) to keep updated with our latest developments.

Together, let's build the future of AI!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>

In [ ]:
# Let's add extra functionality where we translate the brocheure into the language we need. 

def translate_brochure(language):
    system_prompt = f"You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information. Translate the brochure into {language}."
    return system_prompt

In [41]:
print(translate_brochure("Spanish"))

You are an assistant that analyzes the contents of several relevant pages from a company website and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.Include details of company culture, customers and careers/jobs if you have the information. Translate the entire content mentioned in the brochure into Spanish Language.


In [45]:
def stream_brochure_v2(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translate_brochure("Hindi")},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [46]:
stream_brochure_v2("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# 🤗 Hugging Face Brochure 🥳

### Welcome to Hugging Face: The AI Community Building the Future!

At Hugging Face, we're more than just a company; we're a **hugely fun community of super-smart AI enthusiasts**. Think of us as the **Avengers of Machine Learning**—without the capes, but we do have a dazzling array of models, datasets, and applications to save the day!

---

### Why Choose Us? 🤔

- **1 Million+ Models to Play With!** 🎉
  Yes, you read that right! We have enough models to make you feel like a kid in a candy store—only this candy is filled with algorithms and data!

- **Datasets Galore!** 🍭
  Get lost in our collection of **250k+ datasets**! It’s like a treasure hunt, but instead of jewels, you find valuable data nuggets and insights.

- **Spaces That Spark Creativity!** 🪐
  Why just stop at using models when you can create your own apps? We have **12.9k running apps**! That’s enough to make your head spin—like a well-trained AI turning data into gold.

---

### Join Our Community! 👩‍💻👨‍💻

- **Collaborate with the Best!** 🤝
  With over **50,000 organizations** using Hugging Face, we’re practically a UN for AI! Rub shoulders with the likes of Microsoft, Google, and even Grammarly. The only thing missing is your nameplate!

- **Your Work Can Shine!** 🌟
  Build your ML profile and share your stellar work. Show your friends that you can not only model your dreams but also produce them!

---

### Careers at Hugging Face 🌈

- **We Value Happiness!** 😊
  Our culture is all about support and collaboration. We believe in cheerful brainstorming sessions, which sometimes involve actual brainstorming over hot cocoa (or coffee, if that’s your jam).

- **Job Openings!** 🚀
  Looking to join us? We have exciting roles waiting for you—because who wouldn't want to be part of the coolest AI family around?

---

### Comic Corner (aka Fun Facts!) 🤪

- Did you know our models like to “hug” it out? We’re the only models that don’t mind getting cozy!
- If our datasets had a dating profile, it would definitely say “seeking meaningful connections.” 🤭

---

### Let’s Connect! 🌐

Join us on this exciting journey in the AI cosmos! For insights, witty banter, or code chats, reach out to our various social platforms. We promise zero awkwardness—only AI magic!

---

**Hugging Face: Bringing AI to Life with a Big Embrace!** 💖

_Don’t delay, hug today!_


---

### 🤗 हगिंग फेस ब्रोशर 🥳

### हगिंग फेस में आपका स्वागत है: भविष्य का निर्माण करने वाली AI समुदाय!

हगिंग फेस में, हम केवल एक कंपनी नहीं हैं; हम **सुपर-स्मार्ट AI उत्साही लोगों का एक बहुत मजेदार समुदाय** हैं। हमें **मशीन लर्निंग के एवेंजर्स** के रूप में सोचें—केप के बिना, लेकिन हमारे पास बचाने के लिए मॉडल, डेटा सेट और अनुप्रयोगों की एक शानदार श्रृंखला है!

---

### हमें क्यों चुनें? 🤔

- **1 मिलियन+ मॉडल खेलने के लिए!** 🎉
  हाँ, आपने सही पढ़ा! हमारे पास इतने सारे मॉडल हैं कि आप एक कैडी स्टोर में बच्चे की तरह महसूस करेंगे—केवल इस कैंडी में एल्गोरिदम और डेटा भरे हुए हैं!

- **डेटा सेट की बहार!** 🍭
  हमारे **250k+ डेटा सेट** के संग्रह में खो जाएँ! यह एक खजाने की खोज की तरह है, लेकिन अंत में आप डेटा Nuggets और अंतर्दृष्टि पाएंगे।

- **क्रिएटिविटी को प्रेरित करने वाले स्पेस!** 🪐
  क्यों सिर्फ मॉडल का उपयोग करना है जब आप अपने अनुप्रयोग बना सकते हैं? हमारे पास **12.9k चल रहे अनुप्रयोग** हैं! यह हेड स्पिन करने के लिए पर्याप्त है—जैसे एक कुशल AI डेटा को सोने में बदलता है।

---

### हमारे समुदाय में शामिल हों! 👩‍💻👨‍💻

- **सर्वश्रेष्ठ के साथ सहयोग करें!** 🤝
  50,000 से अधिक संगठन हगिंग फेस का उपयोग कर रहे हैं, हम AI के लिए एक प्रकार का यूएन हैं! माइक्रोसॉफ्ट, गूगल, और यहां तक कि ग्रामरली जैसे लोगों के साथ कंधा-से-कंधा मिलाएं। केवल एक चीज गायब है, वह है आपका नामपट्ट!

- **आपका काम चमक सकता है!** 🌟
  अपना ML प्रोफ़ाइल बनाएं और अपने शानदार काम को साझा करें। अपने दोस्तों को दिखाएँ कि आप न केवल अपने सपनों का मॉडल कर सकते हैं बल्कि उन्हें उत्पादित भी कर सकते हैं!

---

### हगिंग फेस में करियर 🌈

- **हम खुशी को महत्व देते हैं!** 😊
  हमारी संस्कृति सहयोग और समर्थन के बारे में है। हम खुशहाल ब्रेनस्टॉर्मिंग सत्रों में विश्वास करते हैं, जिसमें कभी-कभी वास्तविक गर्म कोको पर चर्चा होती है (या कॉफी, अगर वह आपका जाम है)।

- **नौकरी के अवसर!** 🚀
  क्या आप हमारे साथ जुड़ने के लिए उत्सुक हैं? हमारे पास रोमांचक भूमिकाएँ आपके लिए इंतजार कर रही हैं—क्योंकि कौन नहीं चाहता कि वह चारों ओर के सबसे कूल AI परिवार का हिस्सा हो?

---

### कॉमिक कॉर्नर (या मजेदार तथ्य!) 🤪

- क्या आपको पता है कि हमारे मॉडल "हग" करना पसंद करते हैं? हम एकमात्र मॉडल हैं जिन्हें नज़दीकी होने में कोई आपत्ति नहीं है!
- अगर हमारे डेटा सेट का डेटिंग प्रोफ़ाइल होता, तो निश्चित रूप से यह कहता, "गहरे संबंध की तलाश में।" 🤭

---

### आइए जुड़ें! 🌐

AI कॉस्मिक सफर में हमारे साथ जुड़ें! अंतर्दृष्टि, मजेदार बातचीत, या कोड पर चर्चा के लिए, हमारे विभिन्न सोशल प्लेटफार्मों पर पहुँचें। हम वादा करते हैं कि कोई असहजता नहीं होगी—सिर्फ AI जादू!

---

**हगिंग फेस: एक बड़े आलिंगन के साथ AI को जीवन में लाना!** 💖

_आज ही हग न करें, हग करें!_


This brochure captures the essence of Hugging Face with a playful tone to engage prospective customers, investors, and recruits while providing clear information about the company's offerings and culture in both English and Hindi.